In [2]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
import nltk
import re

In [3]:
# import nltk
# nltk.download()

### Stop words / Function words removing.

Data input :

In [4]:
NorthAmerica = pd.read_csv("rawdata/trends/1230/NorthAmerica_Trends1230.csv",sep = ",", header = 0)

NorthAmerica.columns = ['time','tweet','hashtags','location']
# NorthAmerica.columns = ['time','tweet','hashtags','place', 'location']
print len(NorthAmerica.index)
NorthAmerica = NorthAmerica.drop_duplicates(subset='tweet',keep='last') #remove the duplicate tweets
print len(NorthAmerica.index)

TwitterData = NorthAmerica
TwitterData.head()

257813
126125


,time,tweet,hashtags,location
0,Tue Dec 29 23:16:42 +0000 2015,Do you have a favorite color?\r\n#KidQuestions...,KidQuestionsForHillary,NaN
1,Tue Dec 29 23:16:42 +0000 2015,Michigan Tech wins 3-2 in OT! Brent Baltus the...,GLI51,NaN
2,Tue Dec 29 23:16:42 +0000 2015,#MessageForMyCrush love you preeto,MessageForMyCrush,NaN
3,Tue Dec 29 23:16:42 +0000 2015,Stop https://t.co/g3dGd37ZMd,NaN,NaN
4,Tue Dec 29 23:16:42 +0000 2015,#MessageForMyCrush I've picked out my wedding ...,MessageForMyCrush,NaN


### Cleaning, Tokenization and Stemming

Remove stopwords :

we define a function for cleaning the tweets.

In [5]:
#stemming
from nltk.stem.snowball import SnowballStemmer
#from __future__ import print_function, unicode_literals
# Create p_stemmer of class PorterStemmer
s_stemmer = SnowballStemmer("english")

def filter_tweet (sample):
    stopped_tokens = [re.sub(r'(^https?:\/\/.*[\r\n]*)|(@[A-Za-z0-9]+)|(#[A-Za-z0-9]+)|(\n)|([^\w])|(RT)',
                '', word, flags=re.MULTILINE) for word in str(sample).split(" ")] 
#     stopped_tokens = [re.sub(r'(^https?:\/\/.*[\r\n]*)|(@[A-Za-z0-9]+)|(#[A-Za-z0-9]+)|(\n)|([^\w])|(RT)',
#                 '', word, flags=re.MULTILINE) for word in str(sample).split(" ") if word not in stopwords.words('english')] 
    filtered_sample = filter(None,stopped_tokens)
#     filtered_sample = [s_stemmer.stem(i).lower() for i in filtered_sample] 
    #filtered_sample = [s_stemmer(i.lower()) for i in filtered_sample]
    filtered_sample = " ".join(filtered_sample)
    return filtered_sample

### Build the List of tweets :

In [6]:
ListTweets = [filter_tweet(tweet) for tweet in TwitterData.tweet.values]

### Test by sampling :

In [7]:
#sample
sample_index = 5
print TwitterData.tweet.values[sample_index]+"\n\n"
print ListTweets[sample_index]

@Spidiemma the bigger they are the harder they fall eh!!!! https://t.co/IYR1im9toh


the bigger they are the harder they fall eh


In [8]:
print filter_tweet(TwitterData.tweet.values[15])
print TwitterData.tweet.values[15]

Used to telegraph me from your stagecoach
RT @justinjayubo: Used to telegraph me from your stagecoach..  #OldWestPickupLines


In [9]:
#lower for hashtags
ListHashtags =[str(hashtag).lower() for hashtag in TwitterData.hashtags.values]  

### Save the cleaned data to a data frame

In [10]:
TwitterData_cleaned = pd.DataFrame({"tweet": ListTweets, 
                                   "hashtags": ListHashtags,
                                   "location": TwitterData.location})

In [11]:
TwitterData_cleaned.to_csv("predata/NorthAmerica_Trends_cleaned1230_nostop.csv", sep = ",", header = True, index = False)

In [12]:
TwitterData_cleaned[0:10]

,hashtags,location,tweet
0,kidquestionsforhillary,NaN,Do you have a favorite color
1,gli51,NaN,Michigan Tech wins 32 in OT Brent Baltus the h...
2,messageformycrush,NaN,love you preeto
3,nan,NaN,Stop
4,messageformycrush,NaN,Ive picked out my wedding dress your tux and a...
5,nan,"Northern Ireland, United Kingdom",the bigger they are the harder they fall eh
8,nan,NaN,Early round draft prospects to keep an eye on ...
11,pataki,NaN,First time has trended Irony
13,happyvday2015,NaN,GORGEOUS JUST FUCKING GORGEOUS
16,ncaaffr,NaN,Russell Athletic Bowl 1er TD pour les Tar Heel...


# Trends Pre-processing 

We will save two pandas.dataframe to store the 'keywords' and 'hashtags' seprately.

In [13]:
Trends = pd.read_csv("rawdata/trends/1230/Trends1230.csv",sep = ",", header = 0)

Trends.head()

,create,trend
0,2015-12-29T23:15:02Z,Edwin Burgos
1,2015-12-29T23:15:02Z,#OldWestPickupLines
2,2015-12-29T23:15:02Z,#MessageForMyCrush
3,2015-12-29T23:15:02Z,#KidQuestionsForHillary
4,2015-12-29T23:15:02Z,#HelpAllySEE


In [14]:
ListTrends =[str(hashtag).lower() for hashtag in Trends.trend.values]
ListTrends[0:10]

['edwin burgos',
 '#oldwestpickuplines',
 '#messageformycrush',
 '#kidquestionsforhillary',
 '#helpallysee',
 'mike tyson',
 'jared goff',
 '#tyleroakleythankyoufor',
 'baylor',
 'george pataki']

In [15]:
Hashtags_cleaned = pd.DataFrame({"hashtag": [word[1:] for word in ListTrends if "#" in word]})
Hashtags_cleaned = Hashtags_cleaned.drop_duplicates(subset='hashtag',keep='last')
print len(Hashtags_cleaned)
Hashtags_cleaned.head()

49


,hashtag
0,oldwestpickuplines
2,kidquestionsforhillary
3,helpallysee
4,tyleroakleythankyoufor
5,nicksnewvideo


In [16]:
Keywords_cleaned = pd.DataFrame({"keyword": [word for word in ListTrends if "#" not in word]})
Keywords_cleaned = Keywords_cleaned.drop_duplicates(subset='keyword',keep='last')
print len(Keywords_cleaned)
Keywords_cleaned.head()

50


,keyword
0,edwin burgos
1,mike tyson
2,jared goff
3,baylor
4,george pataki


In [17]:
Hashtags_cleaned.to_csv("predata/Hashtags_cleaned1230.csv", sep = ",", header = True, index = False)
Keywords_cleaned.to_csv("predata/Keywords_cleaned1230.csv", sep = ",", header = True, index = False)